In [1]:
#download DOW30 data from yahoo finance
import yfinance as yf
import pandas as pd

#dow30_tickers = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']
dow30_tickers = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT', 'DIS']

dow30 = yf.download(dow30_tickers, start="2010-01-01", end="2021-12-31")

[*********************100%***********************]  29 of 29 completed


In [2]:
dow30 = dow30['Adj Close']

In [3]:
dow30.index = pd.to_datetime(dow30.index)

In [4]:
import numpy as np

In [5]:
def calculate_max_drawdown(portfolio_values):
    max_drawdown = 0
    peak_value = portfolio_values[0]

    for value in portfolio_values:
        if value > peak_value:
            peak_value = value
        else:
            drawdown = (peak_value - value) / peak_value
            max_drawdown = max(max_drawdown, drawdown)

    return max_drawdown

In [6]:
def calculate_sharpe_ratio(portfolio_values, risk_free_rate = 0.0):
    returns = np.diff(portfolio_values) / portfolio_values[:-1]
    average_return = np.mean(returns)
    std_deviation = np.std(returns)
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation
    return sharpe_ratio

In [7]:
dow30.tail(1)

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-30,176.657288,215.582657,160.352356,202.710007,198.327316,255.330002,60.281521,111.264244,155.929993,370.121735,...,73.728905,334.598663,164.696335,155.872482,151.851624,494.255005,215.391678,47.133755,48.307262,139.847702


In [8]:
# Strategy-1 

"""
Dow 30’un tamamını eşit ağırlıklı olacak şekilde al, yani 30 hisse varsa herbirinden 1/30 kadar alın.
"""

data = dow30.copy()

dataframe_1 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    start_value = 0
    end_value = 0
    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'
    temp = data.loc[start_date:end_date]
    for ticket in data.columns:
        start_value += temp[ticket][0]*1/29
        end_value += temp[ticket][-1]*1/29

    annual_return = (end_value - start_value) / start_value
    dataframe_1['Annual Return'][str(i)] = annual_return

    portfolio_values = []
    for day in range(len(temp)):
        portfolio_value = 0
        for ticket in data.columns:
            portfolio_value += temp[ticket][day]*1/29
        portfolio_values.append(portfolio_value)

    dataframe_1['Std Dev'][str(i)] = np.std(portfolio_values)
    dataframe_1['Max Drawdown'][str(i)] = calculate_max_drawdown(portfolio_values)
    dataframe_1['Sharpe Ratio'][str(i)] = calculate_sharpe_ratio(portfolio_values)

dataframe_1['Annual Return']['Average'] = dataframe_1['Annual Return'].mean()
dataframe_1['Std Dev']['Average'] = dataframe_1['Std Dev'].mean()
dataframe_1['Max Drawdown']['Average'] = dataframe_1['Max Drawdown'].mean()
dataframe_1['Sharpe Ratio']['Average'] = dataframe_1['Sharpe Ratio'].mean()

In [9]:
dataframe_1

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio
2015,0.034145,1.998714,0.122733,0.018322
2016,0.153402,4.187928,0.090682,0.074333
2017,0.29195,6.742824,0.036214,0.24522
2018,-0.008359,3.93032,0.172863,0.002775
2019,0.243339,5.596904,0.059939,0.117451
2020,0.094153,12.260769,0.354529,0.027167
2021,0.228297,8.887833,0.06374,0.10982
Average,0.148132,6.229327,0.128671,0.085013


In [10]:
#2 

"""
Bir önceki sene en iyi getiri sağlayan 1 hisseyi alın. Sene boyunca sadece bu hisseyi tutun.
"""

data = dow30.copy()

dataframe_2 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'Best Stock'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    best_stock = None
    best_return = 0

    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]
        if temp_return > best_return:
            best_return = temp_return
            best_stock = stock
    
    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    dataframe_2.loc[str(i), 'Annual Return'] = (data.loc[end_date, best_stock] - data.loc[start_date, best_stock]) / data.loc[start_date, best_stock]

    dataframe_2.loc[str(i), 'Std Dev'] = data.loc[start_date:end_date, best_stock].std()

    dataframe_2.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(data.loc[start_date:end_date, best_stock])

    dataframe_2.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(data.loc[start_date:end_date, best_stock])

    dataframe_2.loc[str(i), 'Best Stock'] = best_stock

dataframe_2.loc['Average', 'Annual Return'] = dataframe_2['Annual Return'].mean()
dataframe_2.loc['Average', 'Std Dev'] = dataframe_2['Std Dev'].mean()
dataframe_2.loc['Average', 'Max Drawdown'] = dataframe_2['Max Drawdown'].mean()
dataframe_2.loc['Average', 'Sharpe Ratio'] = dataframe_2['Sharpe Ratio'].mean()

dataframe_2

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,Best Stock
2015,-0.024022,1.868262,0.283067,0.001378,INTC
2016,-0.163896,3.30549,0.231071,-0.043659,NKE
2017,0.727058,16.085142,0.077055,0.177087,CAT
2018,0.107767,17.179262,0.246703,0.030613,BA
2019,0.235978,3.898046,0.121369,0.078838,MRK
2020,0.782399,21.630771,0.314273,0.092901,AAPL
2021,0.385508,14.558134,0.185989,0.090713,AAPL
Average,0.29297,11.217873,0.208504,0.061124,NaN


In [11]:
# 3

"""
Bir önceki sene en kötü getiri sağlayan 1 hisseyi alın. Sene boyunca sadece bu hisseyi tutun.
"""

data = dow30.copy()

dataframe_3 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'Worst Stock'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    worst_stock = None
    worst_return =  10000000

    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]
        if temp_return < worst_return:
            worst_return = temp_return
            worst_stock = stock
    
    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    dataframe_3.loc[str(i), 'Annual Return'] = (data.loc[end_date, worst_stock] - data.loc[start_date, worst_stock]) / data.loc[start_date, worst_stock]

    dataframe_3.loc[str(i), 'Std Dev'] = data.loc[start_date:end_date, worst_stock].std()

    dataframe_3.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(data.loc[start_date:end_date, worst_stock])

    dataframe_3.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(data.loc[start_date:end_date, worst_stock])

    dataframe_3.loc[str(i), 'Worst Stock'] = worst_stock

dataframe_3.loc['Average', 'Annual Return'] = dataframe_3['Annual Return'].mean()
dataframe_3.loc['Average', 'Std Dev'] = dataframe_3['Std Dev'].mean()
dataframe_3.loc['Average', 'Max Drawdown'] = dataframe_3['Max Drawdown'].mean()
dataframe_3.loc['Average', 'Sharpe Ratio'] = dataframe_3['Sharpe Ratio'].mean()

dataframe_3

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,Worst Stock
2015,-0.123078,7.087031,0.22959,-0.032576,IBM
2016,0.157292,2.937342,0.093002,0.053972,WMT
2017,0.21941,3.07001,0.151901,0.064676,NKE
2018,-0.066457,5.191732,0.247298,-0.007162,WBA
2019,0.364124,12.121987,0.119654,0.091038,GS
2020,-0.29517,4.497888,0.419543,-0.030237,WBA
2021,-0.000049,17.044806,0.300903,0.011111,BA
Average,0.036582,7.421542,0.223127,0.021546,NaN


In [12]:
#4

"""
Bir önceki sene en iyi getiri sağlayan 3 hisseyi alın. Sene boyunca bu 3 hisseyi tutun.
"""

data = dow30.copy()

dataframe_4 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'Best Stocks'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    best_stock_1 = None
    best_return_1 = 0
    best_stock_2 = None
    best_return_2 = 0
    best_stock_3 = None
    best_return_3 = 0

    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]
        if temp_return > best_return_1:
            best_return_3 = best_return_2
            best_stock_3 = best_stock_2
            best_return_2 = best_return_1
            best_stock_2 = best_stock_1
            best_return_1 = temp_return
            best_stock_1 = stock
        elif temp_return > best_return_2:
            best_return_3 = best_return_2
            best_stock_3 = best_stock_2
            best_return_2 = temp_return
            best_stock_2 = stock
        elif temp_return > best_return_3:
            best_return_3 = temp_return
            best_stock_3 = stock
        
    best_stocks = [best_stock_1, best_stock_2, best_stock_3]

    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    temp_data = data.loc[start_date:end_date, best_stocks]
    temp_data['Portfolio'] = temp_data.sum(axis=1) / 3
    portfolio_values = temp_data['Portfolio'].values

    dataframe_4.loc[str(i), 'Annual Return'] = (temp_data['Portfolio'][-1] - temp_data['Portfolio'][0]) / temp_data['Portfolio'][0]
    dataframe_4.loc[str(i), 'Std Dev'] = temp_data['Portfolio'].std()
    dataframe_4.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(portfolio_values)
    dataframe_4.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(portfolio_values)

    dataframe_4.loc[str(i), 'Best Stocks'] = best_stocks

dataframe_4.loc['Average', 'Annual Return'] = dataframe_4['Annual Return'].mean()
dataframe_4.loc['Average', 'Std Dev'] = dataframe_4['Std Dev'].mean()
dataframe_4.loc['Average', 'Max Drawdown'] = dataframe_4['Max Drawdown'].mean()
dataframe_4.loc['Average', 'Sharpe Ratio'] = dataframe_4['Sharpe Ratio'].mean()

dataframe_4

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,Best Stocks
2015,0.018433,2.184457,0.191996,0.012351,"[INTC, AAPL, AMGN]"
2016,-0.047554,2.787036,0.138621,-0.012481,"[NKE, CRM, MCD]"
2017,0.44492,13.179917,0.044739,0.207552,"[CAT, UNH, JPM]"
2018,0.008671,8.099056,0.260453,0.01088,"[BA, CAT, AAPL]"
2019,0.332757,6.834985,0.080944,0.10399,"[MRK, CRM, MSFT]"
2020,0.313131,15.767894,0.330314,0.054102,"[AAPL, MSFT, JPM]"
2021,0.416748,21.185829,0.089728,0.123757,"[AAPL, MSFT, NKE]"
Average,0.212444,10.005596,0.162399,0.07145,NaN


In [14]:
#5

"""
Bir önceki sene en kötü getiri sağlayan 3 hisseyi alın. Sene boyunca bu 3 hisseyi tutun.
"""


data = dow30.copy()

dataframe_5 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'worst Stocks'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    worst_stock_1 = None
    worst_return_1 = 100000000
    worst_stock_2 = None
    worst_return_2 = 100000000
    worst_stock_3 = None
    worst_return_3 = 100000000

    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]

        if temp_return < worst_return_1:
            worst_return_3 = worst_return_2
            worst_stock_3 = worst_stock_2
            worst_return_2 = worst_return_1
            worst_stock_2 = worst_stock_1
            worst_return_1 = temp_return
            worst_stock_1 = stock   
        elif temp_return < worst_return_2:
            worst_return_3 = worst_return_2
            worst_stock_3 = worst_stock_2
            worst_return_2 = temp_return
            worst_stock_2 = stock  
        elif temp_return < worst_return_3:
            worst_return_3 = temp_return
            worst_stock_3 = stock
            


    worst_stocks = [worst_stock_1, worst_stock_2, worst_stock_3]

    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    temp_data = data.loc[start_date:end_date, worst_stocks]
    temp_data['Portfolio'] = temp_data.sum(axis=1) / 3
    portfolio_values = temp_data['Portfolio'].values

    dataframe_5.loc[str(i), 'Annual Return'] = (temp_data['Portfolio'][-1] - temp_data['Portfolio'][0]) / temp_data['Portfolio'][0]
    dataframe_5.loc[str(i), 'Std Dev'] = temp_data['Portfolio'].std()
    dataframe_5.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(portfolio_values)
    dataframe_5.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(portfolio_values)

    dataframe_5.loc[str(i), 'worst Stocks'] = worst_stocks

dataframe_5.loc['Average', 'Annual Return'] = dataframe_5['Annual Return'].mean()
dataframe_5.loc['Average', 'Std Dev'] = dataframe_5['Std Dev'].mean()
dataframe_5.loc['Average', 'Max Drawdown'] = dataframe_5['Max Drawdown'].mean()
dataframe_5.loc['Average', 'Sharpe Ratio'] = dataframe_5['Sharpe Ratio'].mean()

dataframe_5

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,worst Stocks
2015,-0.033465,5.207773,0.206685,-0.005931,"[IBM, CVX, BA]"
2016,0.229387,5.016865,0.100302,0.085422,"[WMT, AXP, CAT]"
2017,0.269006,5.629195,0.082594,0.131135,"[NKE, CRM, AMGN]"
2018,-0.054153,2.87518,0.174373,-0.012481,"[WBA, IBM, MRK]"
2019,0.27688,7.128019,0.131121,0.085296,"[GS, IBM, CAT]"
2020,-0.243303,24.225671,0.547308,-0.014375,"[WBA, MMM, BA]"
2021,0.146325,6.957284,0.200649,0.040787,"[BA, WBA, CVX]"
Average,0.084383,8.14857,0.206147,0.044265,NaN


In [15]:
#6

"""
Bir önceki sene en iyi getiri sağlayan 5 hisseyi alın. Sene boyunca bu 5 hisseyi tutun.
"""


#4

"""
Bir önceki sene en iyi getiri sağlayan 3 hisseyi alın. Sene boyunca bu 3 hisseyi tutun.
"""

data = dow30.copy()

dataframe_6 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'Best Stocks'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    best_stock_1 = None
    best_return_1 = 0
    best_stock_2 = None
    best_return_2 = 0
    best_stock_3 = None
    best_return_3 = 0
    best_return_4 = 0
    best_stock_4 = None
    best_return_5 = 0
    best_stock_5 = None


    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]
        
        if temp_return > best_return_1:
            best_stock_5 = best_stock_4
            best_return_5 = best_return_4
            best_stock_4 = best_stock_3
            best_return_4 = best_return_3
            best_stock_3 = best_stock_2
            best_return_3 = best_return_2
            best_stock_2 = best_stock_1
            best_return_2 = best_return_1
            best_stock_1 = stock
            best_return_1 = temp_return
        elif temp_return > best_return_2:
            best_stock_5 = best_stock_4
            best_return_5 = best_return_4
            best_stock_4 = best_stock_3
            best_return_4 = best_return_3
            best_stock_3 = best_stock_2
            best_return_3 = best_return_2
            best_stock_2 = stock
            best_return_2 = temp_return
        elif temp_return > best_return_3:
            best_stock_5 = best_stock_4
            best_return_5 = best_return_4
            best_stock_4 = best_stock_3
            best_return_4 = best_return_3
            best_stock_3 = stock
            best_return_3 = temp_return
        elif temp_return > best_return_4:
            best_stock_5 = best_stock_4
            best_return_5 = best_return_4
            best_stock_4 = stock
            best_return_4 = temp_return
        elif temp_return > best_return_5:
            best_stock_5 = stock
            best_return_5 = temp_return
            
        
    best_stocks = [best_stock_1, best_stock_2, best_stock_3, best_stock_4, best_stock_5]

    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    temp_data = data.loc[start_date:end_date, best_stocks]
    temp_data['Portfolio'] = temp_data.sum(axis=1) / 5
    portfolio_values = temp_data['Portfolio'].values

    dataframe_6.loc[str(i), 'Annual Return'] = (temp_data['Portfolio'][-1] - temp_data['Portfolio'][0]) / temp_data['Portfolio'][0]
    dataframe_6.loc[str(i), 'Std Dev'] = temp_data['Portfolio'].std()
    dataframe_6.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(portfolio_values)
    dataframe_6.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(portfolio_values)

    dataframe_6.loc[str(i), 'Best Stocks'] = best_stocks

dataframe_6.loc['Average', 'Annual Return'] = dataframe_6['Annual Return'].mean()
dataframe_6.loc['Average', 'Std Dev'] = dataframe_6['Std Dev'].mean()
dataframe_6.loc['Average', 'Max Drawdown'] = dataframe_6['Max Drawdown'].mean()
dataframe_6.loc['Average', 'Sharpe Ratio'] = dataframe_6['Sharpe Ratio'].mean()

dataframe_6


,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,Best Stocks
2015,0.085736,2.381635,0.148653,0.031155,"[INTC, AAPL, AMGN, UNH, WBA]"
2016,0.005114,2.490972,0.135158,0.007039,"[NKE, CRM, MCD, HD, MSFT]"
2017,0.260755,9.863159,0.082038,0.128482,"[CAT, UNH, JPM, CVX, GS]"
2018,0.012598,5.213107,0.186565,0.010606,"[BA, CAT, AAPL, MCD, WMT]"
2019,0.363331,7.854863,0.075208,0.124294,"[MRK, CRM, MSFT, NKE, V]"
2020,0.239841,14.149819,0.321835,0.047742,"[AAPL, MSFT, JPM, V, PG]"
2021,0.299693,16.509161,0.079135,0.107291,"[AAPL, MSFT, NKE, CRM, CAT]"
Average,0.18101,8.351817,0.146942,0.06523,NaN


In [16]:
#7

"""
Bir önceki sene en kötü getiri sağlayan 5 hisseyi alın. Sene boyunca bu 5 hisseyi tutun.
"""


data = dow30.copy()

dataframe_7 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'worst Stocks'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    worst_stock_1 = None
    worst_return_1 = 100000000
    worst_stock_2 = None
    worst_return_2 = 100000000
    worst_stock_3 = None
    worst_return_3 = 100000000
    worst_return_4 = 100000000
    worst_stock_4 = None
    worst_return_5 = 100000000
    worst_stock_5 = None


    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]

        if temp_return < worst_return_1:
            worst_return_5 = worst_return_4
            worst_stock_5 = worst_stock_4
            worst_return_4 = worst_return_3
            worst_stock_4 = worst_stock_3
            worst_return_3 = worst_return_2
            worst_stock_3 = worst_stock_2
            worst_return_2 = worst_return_1
            worst_stock_2 = worst_stock_1
            worst_return_1 = temp_return
            worst_stock_1 = stock
        elif temp_return < worst_return_2:
            worst_return_5 = worst_return_4
            worst_stock_5 = worst_stock_4
            worst_return_4 = worst_return_3
            worst_stock_4 = worst_stock_3
            worst_return_3 = worst_return_2
            worst_stock_3 = worst_stock_2
            worst_return_2 = temp_return
            worst_stock_2 = stock
        elif temp_return < worst_return_3:
            worst_return_5 = worst_return_4
            worst_stock_5 = worst_stock_4
            worst_return_4 = worst_return_3
            worst_stock_4 = worst_stock_3
            worst_return_3 = temp_return
            worst_stock_3 = stock
        elif temp_return < worst_return_4:
            worst_return_5 = worst_return_4
            worst_stock_5 = worst_stock_4
            worst_return_4 = temp_return
            worst_stock_4 = stock
        elif temp_return < worst_return_5:
            worst_return_5 = temp_return
            worst_stock_5 = stock
            


    worst_stocks = [worst_stock_1, worst_stock_2, worst_stock_3, worst_stock_4, worst_stock_5]

    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    temp_data = data.loc[start_date:end_date, worst_stocks]
    temp_data['Portfolio'] = temp_data.sum(axis=1) / 5
    portfolio_values = temp_data['Portfolio'].values

    dataframe_7.loc[str(i), 'Annual Return'] = (temp_data['Portfolio'][-1] - temp_data['Portfolio'][0]) / temp_data['Portfolio'][0]
    dataframe_7.loc[str(i), 'Std Dev'] = temp_data['Portfolio'].std()
    dataframe_7.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(portfolio_values)
    dataframe_7.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(portfolio_values)

    dataframe_7.loc[str(i), 'worst Stocks'] = worst_stocks

dataframe_7.loc['Average', 'Annual Return'] = dataframe_7['Annual Return'].mean()
dataframe_7.loc['Average', 'Std Dev'] = dataframe_7['Std Dev'].mean()
dataframe_7.loc['Average', 'Max Drawdown'] = dataframe_7['Max Drawdown'].mean()
dataframe_7.loc['Average', 'Sharpe Ratio'] = dataframe_7['Sharpe Ratio'].mean()

dataframe_7

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,worst Stocks
2015,0.035274,3.1515,0.171908,0.018352,"[IBM, CVX, BA, VZ, MCD]"
2016,0.270852,6.190072,0.099316,0.104263,"[WMT, AXP, CAT, CVX, IBM]"
2017,0.183335,3.170514,0.064922,0.109739,"[NKE, CRM, AMGN, KO, WBA]"
2018,-0.022288,3.142278,0.145381,-0.003112,"[WBA, IBM, MRK, VZ, DIS]"
2019,0.170149,5.366842,0.147509,0.060956,"[GS, IBM, CAT, MMM, CVX]"
2020,-0.228514,16.481644,0.515376,-0.017062,"[WBA, MMM, BA, CVX, VZ]"
2021,0.119787,4.50999,0.169602,0.039761,"[BA, WBA, CVX, INTC, MRK]"
Average,0.075514,6.001834,0.187716,0.044699,NaN


In [ ]:
#8